# Ejemplo Particionado de Datos

In [0]:
import analytics.general_utils as gen_utils
import time
import pyspark.sql.functions as F

## Sin Particionar

In [0]:
# (spark
#  .read
#  .parquet("/mnt/analytics/test_particionado/fp_001_part")
#  .repartition(44*58)
#  .write
#  .parquet("/mnt/analytics/test_particionado/fp_001_no_part_def"))

In [0]:
# Leemos tabla inicial sin particionar. Contiene datos de 44 particiones

fp_001 = (spark.read.parquet("/mnt/analytics/test_particionado/fp_001_no_part_def"))

In [0]:
%%timeit -n 3 -r 1

# Vemos lo que tarda en hacer un count & distinct sin ningún filtro
(fp_001
 .distinct()
 .count())

3 loops, best of 1: 1min 4s per loop

In [0]:
%%timeit -n 3 -r 1

# Vemos lo que tarda en hacer un count & distinct filtrando por una partición
(fp_001
 .filter(F.col("periodo") == 20200610)
 .distinct()
 .count())

3 loops, best of 1: 33.9 s per loop

## Particionado por periodo: PARQUET

In [0]:
# Escribimos, pero particionando por periodo

# (fp_001
#  .write
#  .mode("OVERWRITE")
#  .partitionBy("periodo")
#  .parquet("/mnt/analytics/test_particionado/fp_001_part"))

In [0]:
# Leemos tabla particionada, todas las particiones

fp_001_all_parts = (spark
                   .read
                   .parquet("/mnt/analytics/test_particionado/fp_001_part"))

In [0]:
%%timeit -n 3 -r 1

# Vemos lo que tarda en hacer un count & distinct sin ningún filtro (antes 1m 4s)
(fp_001_all_parts
 .distinct()
 .count())

3 loops, best of 1: 49.3 s per loop

In [0]:
%%timeit -n 3 -r 1

# Vemos lo que tarda en hacer un count & distinct filtrando por una partición (antes 33.9s)
(fp_001_all_parts
 .filter(F.col("periodo") == 20200610)
 .distinct()
 .count())

3 loops, best of 1: 2.44 s per loop

In [0]:
# Leemos de manera individual una única partición

fp_001_20200610 = (spark
                   .read
                   .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200610"))

In [0]:
%%timeit -n 3 -r 1

# Vemos lo que tarda en hacer un count & distinct "filtrando por una partición" (antes 2.44s)

(fp_001_20200610
 .distinct()
 .count())

3 loops, best of 1: 2.42 s per loop

# Particionado por periodo: DELTA

In [0]:
# Escribimos en formato delta particionado por periodo

# (fp_001
#  .write
#  .format("DELTA")
#  .mode("OVERWRITE")
#  .partitionBy("periodo")
#  .save("/mnt/analytics/test_particionado/fp_001_part_delta"))

In [0]:
# Leemos tabla en formato Delta

fp_001_all_parts_delta = (spark
                          .read
                          .format("delta")
                          .load("/mnt/analytics/test_particionado/fp_001_part_delta"))

In [0]:
%%timeit -n 3 -r 1

# Vemos lo que tarda en hacer un count & distinct filtrando por una partición (antes 2.44s)
(fp_001_all_parts_delta
 .filter(F.col("periodo") == 20200610)
 .distinct()
 .count())

3 loops, best of 1: 2.72 s per loop

# Varios niveles de particionado & DELTA OPTIMIZE

In [0]:
# Escribimos en formato delta particionado por periodo y racepint

(fp_001
 .write
 .format("DELTA")
 .mode("OVERWRITE")
 .partitionBy(["periodo", "racepint"])
 .save("/mnt/analytics/test_particionado/fp_001_part_delta_2"))

In [0]:
# Query para la optimización de la escritura de la tabla

# %sql
# OPTIMIZE delta.`/mnt/analytics/test_particionado/fp_001_part_delta_2`

In [0]:
# Leemos tabla en formato delta con doble partición

fp_001_all_parts_delta_2 = (spark
                          .read
                          .format("delta")
                          .load("/mnt/analytics/test_particionado/fp_001_part_delta_2"))

In [0]:
%%timeit -n 10 -r 1

# Vemos lo que tardaba con una partición un conteo & distinct aplicando 2 filtros

(fp_001_all_parts_delta
 .filter((F.col("periodo") == 20200610) & 
         (F.col("racepint") == "S"))
 .distinct()
 .count())

10 loops, best of 1: 3.05 s per loop

In [0]:
%%timeit -n 10 -r 1

# Vemos lo que tarda ahora con doble partición un conteo & distinct aplicando 2 filtros
(fp_001_all_parts_delta_2
 .filter((F.col("periodo") == 20200610) & 
         (F.col("racepint") == "S"))
 .distinct()
 .count())

10 loops, best of 1: 1.05 s per loop

# Sobreescritura de particiones

## Utilizando Delta Lake

In [0]:
# Sacamos primero el conteo de la partición original que queremos sustituir

(fp_001_all_parts_delta_2
 .filter(F.col("periodo") == 20200501)
 .count())

Out[ 303 ]: 4409136

In [0]:
# Conteo de la tabla total

(fp_001_all_parts_delta_2.count())

Out[ 304 ]: 194027547

In [0]:
# Obtenemos datos nuevos para la partición 20200501

new_part_20200501 = (spark
                     .read
                     .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200501")
                     .sample(0.01)
                     .withColumn("periodo", F.lit("20200501").cast("int")))

In [0]:
new_part_20200501.cache()
new_part_20200501.count()

Out[ 306 ]: 44478

In [0]:
# Sobreescribimos los datos únicamente de la partición 20200501, conservando el esquema y 
# el particionado inicial

(new_part_20200501
 .write
 .format("DELTA")
 .mode("OVERWRITE")
 .option("replaceWhere", "periodo = 20200501")
 .option("mergeSchema", "true")
 .save("/mnt/analytics/test_particionado/fp_001_part_delta_2"))

In [0]:
# Leemos nueva tabla

fp_001_all_parts_delta_2_new = (spark
                                .read
                                .format("DELTA")
                                .load("/mnt/analytics/test_particionado/fp_001_part_delta_2"))

In [0]:
# 194027547, 4409136, 44478
print("Conteo inicial de la tabla: 194027547")
print("Conteo de la partición original de la tabla: 4409136")
print("Conteo de la partición nueva de la tabla: 44478")
print("Conteo de la tabla modificada: {0}".format(194027547-4409136+44478))

Conteo inicial de la tabla: 194027547
Conteo de la partición original de la tabla: 4409136
Conteo de la partición nueva de la tabla: 44478
Conteo de la tabla modificada: 189662889

In [0]:
# Conteo de la nueva partición

(fp_001_all_parts_delta_2_new
 .filter(F.col("periodo") == 20200501)
 .count())

Out[ 311 ]: 44478

In [0]:
# Conteo de la tabla final

fp_001_all_parts_delta_2_new.count()

Out[ 313 ]: 189662889

In [0]:
# Añadimos una nueva partición 20210501 con los datos de la anterior

In [0]:
# Añadimos una nueva partición 20210501 con los datos de la anterior
# Creamos datos de prueba

new_part_20210501 = (spark
                     .read
                     .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200501")
                     .sample(0.01)
                     .withColumn("periodo", F.lit("20210501").cast("int")))

In [0]:
new_part_20210501.cache()
new_part_20210501.count()

Out[ 322 ]: 43933

In [0]:
# Escribmos datos sobre una partición inexistente

(new_part_20210501
 .write
 .format("DELTA")
 .mode("OVERWRITE")
 .option("replaceWhere", "periodo = 20210501")
 .option("mergeSchema", "true")
 .save("/mnt/analytics/test_particionado/fp_001_part_delta_2"))

In [0]:
# Leemos nueva tabla

fp_001_all_parts_delta_2_new = (spark
                                .read
                                .format("DELTA")
                                .load("/mnt/analytics/test_particionado/fp_001_part_delta_2"))

In [0]:
(fp_001_all_parts_delta_2_new
 .filter(F.col("periodo") == 20210501)
 .count())

Out[ 325 ]: 43933

In [0]:
print("El conteo de la nueva tabla debería de ser: {0}".format(189662889+43933))

El conteo de la nueva tabla debería de ser: 189706822

In [0]:
fp_001_all_parts_delta_2_new.count()

Out[ 326 ]: 189706822

## Utilizando formato Parquet convencional

In [0]:
# A continuación se escribe como sería realizar la misma operación del apartado anterior pero utilizando formato 
# parquet convencional


# (new_part_20200501
#  .filter(F.col("racepint")=="N")
#  .drop("periodo", "racepint")
#  .write
#  .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200501/racepint==N"))


# (new_part_20200501
#  .filter(F.col("racepint")=="S")
#  .drop("periodo", "racepint")
#  .write
#  .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200501/racepint==S))

# Escritura de nuevas particiones y sobre-escritura de antiguas con un mismo ccomando

In [0]:
# Creamos datos para la partición: 20210501

new_part_20210501 = (spark
                     .read
                     .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200501")
                     .sample(0.01)
                     .withColumn("periodo", F.lit("20210501").cast("int")))

new_part_20210501.cache()
new_part_20210501.count()

Out[ 330 ]: 44234

In [0]:
# Creamos datos para la partición: 20220501

new_part_20220501 = (spark
                     .read
                     .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200501")
                     .sample(0.02)
                     .withColumn("periodo", F.lit("20220501").cast("int")))

new_part_20220501.cache()
new_part_20220501.count()


Out[ 331 ]: 87714

In [0]:
# Persistimos partición 20210501

(new_part_20210501
 .write
 .format("DELTA")
 .mode("OVERWRITE")
 .option("replaceWhere", "periodo = 20210501")
 .partitionBy(["periodo", "racepint"])
 .save("/mnt/analytics/test_particionado/fp_001_part_delta_3"))

In [0]:
# Persistimos partición 20220501

(new_part_20220501
 .write
 .format("DELTA")
 .mode("OVERWRITE")
 .option("replaceWhere", "periodo = 20220501")
 .partitionBy(["periodo", "racepint"])
 .save("/mnt/analytics/test_particionado/fp_001_part_delta_3"))

In [0]:
# Leemos nueva tabla

fp_001_part_delta_3 = (spark
                       .read
                       .format("DELTA")
                       .load("/mnt/analytics/test_particionado/fp_001_part_delta_3"))

In [0]:
# Comprobamos conteos

(fp_001_part_delta_3
 .groupBy("periodo")
 .count()
 .show())

+--------+-----+
 periodo|count|
+--------+-----+
20210501|43933|
20220501|87714|
+--------+-----+

In [0]:
# Nuevos datos para la partición 20220501

new_part_20220501 = (spark
                     .read
                     .parquet("/mnt/analytics/test_particionado/fp_001_part/periodo=20200501")
                     .sample(0.05)
                     .withColumn("periodo", F.lit("20220501").cast("int")))

new_part_20220501.cache()
new_part_20220501.count()

Out[ 337 ]: 221178

In [0]:
# Sustituimos partición 20220501

(new_part_20220501
 .write
 .format("DELTA")
 .mode("OVERWRITE")
 .option("replaceWhere", "periodo = 20220501")
 .partitionBy(["periodo", "racepint"])
 .save("/mnt/analytics/test_particionado/fp_001_part_delta_3"))

In [0]:
fp_001_part_delta_3 = (spark
                       .read
                       .format("DELTA")
                       .load("/mnt/analytics/test_particionado/fp_001_part_delta_3"))

In [0]:
# Comproabmos conteos

(fp_001_part_delta_3
 .groupBy("periodo")
 .count()
 .show())

+--------+------+
 periodo| count|
+--------+------+
20210501| 43933|
20220501|221178|
+--------+------+

# Conclusiones finales

Se recomienda utilizar formato `DELTA` con varios niveles de particionado

In [0]:
# periodo

In [0]:
df 

In [0]:
# COMANDO BASE

path_table = ""

(df # df es la tabla
 .write
 .format("DELTA")
 .mode("OVERWRITE")
 .option("replaceWhere", "periodo = {0}".format(periodo)) # periodo es el periodo de la ta
 .partitionBy(["periodo"])
 .save(path_table))

In [0]:
# COMANDO BASE

# (new_part_20220501
#  .write
#  .format("DELTA")
#  .mode("OVERWRITE")
#  .option("replaceWhere", "periodo = 20220501")
#  .partitionBy(["periodo", "racepint"])
#  .save("/mnt/analytics/test_particionado/fp_001_part_delta_3"))